<a href="https://colab.research.google.com/github/samitha278/gpt2-lite/blob/main/repro_gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

from dataclasses import dataclass

from transformers import GPT2LMHeadModel , pipeline , set_seed


In [51]:
model = GPT2LMHeadModel.from_pretrained("gpt2")
params_dict = model.state_dict()

In [52]:
real_d = {}
for k,v, in params_dict.items():
   real_d[k] = v.shape

In [53]:
for i, (k, v) in enumerate(params_dict.items()):
    if i < 20 or i > len(params_dict) - 5:
        print(k, v.shape)

transformer.wte.weight torch.Size([50257, 768])
transformer.wpe.weight torch.Size([1024, 768])
transformer.h.0.ln_1.weight torch.Size([768])
transformer.h.0.ln_1.bias torch.Size([768])
transformer.h.0.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.0.attn.c_attn.bias torch.Size([2304])
transformer.h.0.attn.c_proj.weight torch.Size([768, 768])
transformer.h.0.attn.c_proj.bias torch.Size([768])
transformer.h.0.ln_2.weight torch.Size([768])
transformer.h.0.ln_2.bias torch.Size([768])
transformer.h.0.mlp.c_fc.weight torch.Size([768, 3072])
transformer.h.0.mlp.c_fc.bias torch.Size([3072])
transformer.h.0.mlp.c_proj.weight torch.Size([3072, 768])
transformer.h.0.mlp.c_proj.bias torch.Size([768])
transformer.h.1.ln_1.weight torch.Size([768])
transformer.h.1.ln_1.bias torch.Size([768])
transformer.h.1.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.1.attn.c_attn.bias torch.Size([2304])
transformer.h.1.attn.c_proj.weight torch.Size([768, 768])
transformer.h.1.attn.c_proj.bias 

In [54]:
set_seed(220064)
gen = pipeline('text-generation' , model = 'gpt2')
prompt = "Who is the first president is sri lanka?"
gen(prompt,max_new_tokens= 10 , max_length=30)

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=30) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'generated_text': 'Who is the first president is sri lanka? and jeeves! We are a huge country'}]

### test

In [55]:
temp = torch.tensor([
    [[1,2,3],
    [4,5,6]],

    [[7,8,9],
    [12,11,10]],

    ])

print(temp@temp.transpose(-2,-1))

tensor([[[ 14,  32],
         [ 32,  77]],

        [[194, 262],
         [262, 365]]])


### Configurations

In [56]:
@dataclass
class GPT2Config:
  batch_size = 32
  block_size = 8
  n_head = 4
  n_layer = 6
  n_embd = 128

## Attention

In [57]:
class SelfAttentionHead(nn.Module):


  def __init__(self,config):
    super().__init__()

    block_size = config.block_size
    n_embd = config.n_embd
    n_head = config.n_head
    head_size = n_embd // n_head

    self.head_size = head_size


    self.key = nn.Linear(n_embd,head_size)
    self.query = nn.Linear(n_embd,head_size)
    self.value = nn.Linear(n_embd,head_size)
    self.register_buffer('tril' , torch.tril(torch.ones(block_size,block_size)))



  def forward(self,x):

    key = self.key(x)
    query = self.query(x)

    weight = (query @ key.transpose(-1,-2)  ) * (self.head_size**-0.5)
    weight = weight.masked_fill(self.tril[:]==0,float('-inf'))
    weight = F.softmax(weight,dim = -1)

    value = self.value(x)

    out = weight @ value

    return out



In [58]:
class MultiHead(nn.Module):

  def __init__(self,config):
    super().__init__()

    n_head = config.n_head
    n_embd = config.n_embd


    self.self_attns = nn.ModuleList([SelfAttentionHead(config)  for i in range(n_head)])
    self.projection = nn.Linear(n_embd,n_embd)


  def forward(self,x):

    out = torch.cat([sa(x) for sa in self.self_attns],dim=-1)
    out = self.projection(out)

    return out





In [59]:
config = GPT2Config()
h = MultiHead(config)
dic = h.state_dict()
print(h)
for k,v, in dic.items():
  print(k,v.shape)

MultiHead(
  (self_attns): ModuleList(
    (0-3): 4 x SelfAttentionHead(
      (key): Linear(in_features=128, out_features=32, bias=True)
      (query): Linear(in_features=128, out_features=32, bias=True)
      (value): Linear(in_features=128, out_features=32, bias=True)
    )
  )
  (projection): Linear(in_features=128, out_features=128, bias=True)
)
self_attns.0.tril torch.Size([8, 8])
self_attns.0.key.weight torch.Size([32, 128])
self_attns.0.key.bias torch.Size([32])
self_attns.0.query.weight torch.Size([32, 128])
self_attns.0.query.bias torch.Size([32])
self_attns.0.value.weight torch.Size([32, 128])
self_attns.0.value.bias torch.Size([32])
self_attns.1.tril torch.Size([8, 8])
self_attns.1.key.weight torch.Size([32, 128])
self_attns.1.key.bias torch.Size([32])
self_attns.1.query.weight torch.Size([32, 128])
self_attns.1.query.bias torch.Size([32])
self_attns.1.value.weight torch.Size([32, 128])
self_attns.1.value.bias torch.Size([32])
self_attns.2.tril torch.Size([8, 8])
self_attns

In [60]:
sa = SelfAttentionHead(config)
dic = sa.state_dict()

print(sa)
for k,v, in dic.items():
  print(k,v.shape)

SelfAttentionHead(
  (key): Linear(in_features=128, out_features=32, bias=True)
  (query): Linear(in_features=128, out_features=32, bias=True)
  (value): Linear(in_features=128, out_features=32, bias=True)
)
tril torch.Size([8, 8])
key.weight torch.Size([32, 128])
key.bias torch.Size([32])
query.weight torch.Size([32, 128])
query.bias torch.Size([32])
value.weight torch.Size([32, 128])
value.bias torch.Size([32])


In [61]:
x = torch.randn((32,8,128))
sa(x).shape

torch.Size([32, 8, 32])

In [62]:
h(x).shape

torch.Size([32, 8, 128])

## Self Attention Head class + Multi Head class -> Single class

In [63]:
torch.manual_seed(278)
out = torch.randint(5,(32,4,4,3))
out[0][0],out[0][1],out[0][2],out[0][3]

(tensor([[0, 4, 0],
         [0, 1, 1],
         [0, 0, 4],
         [4, 2, 4]]),
 tensor([[4, 0, 4],
         [0, 2, 4],
         [2, 2, 4],
         [3, 2, 2]]),
 tensor([[0, 2, 4],
         [2, 3, 3],
         [0, 1, 4],
         [1, 4, 4]]),
 tensor([[3, 1, 1],
         [3, 1, 4],
         [3, 4, 4],
         [3, 0, 3]]))

In [64]:
B,nh,T,C = out.shape
out = out.permute(0,2,1,3)
out = out.reshape(B,T,nh*C)
out[0]

tensor([[0, 4, 0, 4, 0, 4, 0, 2, 4, 3, 1, 1],
        [0, 1, 1, 0, 2, 4, 2, 3, 3, 3, 1, 4],
        [0, 0, 4, 2, 2, 4, 0, 1, 4, 3, 4, 4],
        [4, 2, 4, 3, 2, 2, 1, 4, 4, 3, 0, 3]])

In [65]:
key = nn.ModuleList(nn.Linear(128,32) for _ in range(4))
x = torch.randn((32,8,128))
out1 = torch.stack([k(x) for k in key],dim=1)
out2 = torch.stack([k(x) for k in key],dim=1)
out = out1 @ out2.transpose(-1,-2)
out.shape

torch.Size([32, 4, 8, 8])

In [66]:
class Attention(nn.Module):


  def __init__(self,config):
    super().__init__()

    block_size = config.block_size
    n_embd = config.n_embd
    n_head = config.n_head
    head_size = n_embd // n_head
    self.head_size = head_size



    self.key = nn.ModuleList(nn.Linear(n_embd,head_size) for _ in range(n_head))
    self.query = nn.ModuleList(nn.Linear(n_embd,head_size) for _ in range(n_head))
    self.value = nn.ModuleList(nn.Linear(n_embd,head_size) for _ in range(n_head))

    self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size)))


  def forward(self,x):


    key = torch.stack([k(x) for k in self.key],dim=1)
    query = torch.stack([q(x) for q in self.query],dim=1)

    weight = query @ key.transpose(-1,-2)
    weight = weight.masked_fill(self.tril[:]==0,float('-inf'))
    weight = F.softmax(weight,dim=-1)

    value = torch.stack([v(x) for v in self.value],dim=1)

    out = weight @ value

    B,nh,T,C = out.shape
    out = out.permute(0,2,1,3)
    out = out.reshape(B,T,nh*C)

    return out

In [67]:
config = GPT2Config()
h = Attention(config)
dic = h.state_dict()
print(h)
for k,v, in dic.items():
  print(k,v.shape)

Attention(
  (key): ModuleList(
    (0-3): 4 x Linear(in_features=128, out_features=32, bias=True)
  )
  (query): ModuleList(
    (0-3): 4 x Linear(in_features=128, out_features=32, bias=True)
  )
  (value): ModuleList(
    (0-3): 4 x Linear(in_features=128, out_features=32, bias=True)
  )
)
tril torch.Size([8, 8])
key.0.weight torch.Size([32, 128])
key.0.bias torch.Size([32])
key.1.weight torch.Size([32, 128])
key.1.bias torch.Size([32])
key.2.weight torch.Size([32, 128])
key.2.bias torch.Size([32])
key.3.weight torch.Size([32, 128])
key.3.bias torch.Size([32])
query.0.weight torch.Size([32, 128])
query.0.bias torch.Size([32])
query.1.weight torch.Size([32, 128])
query.1.bias torch.Size([32])
query.2.weight torch.Size([32, 128])
query.2.bias torch.Size([32])
query.3.weight torch.Size([32, 128])
query.3.bias torch.Size([32])
value.0.weight torch.Size([32, 128])
value.0.bias torch.Size([32])
value.1.weight torch.Size([32, 128])
value.1.bias torch.Size([32])
value.2.weight torch.Size([32

In [68]:
x = torch.randn((32,8,128))

In [69]:
out = h(x)

In [70]:
out.shape

torch.Size([32, 8, 128])

## Attention version

In [71]:
class SelfAttention(nn.Module):

    def __init__(self, config):
        super().__init__()

        block_size = config.block_size

        self.n_head = n_embd = config.n_head
        self.n_embd = n_head = config.n_embd

        assert n_embd % n_head == 0
        self.head_size = n_embd // n_head

        self.c_attn = nn.Linear(n_embd, 3 * n_embd)     # fan out : n_head * 3 * head_size

        self.c_proj = nn.Linear(n_embd, n_embd)

        self.register_buffer("bias", torch.tril(torch.ones(block_size, block_size)))



    def forward(self, x):
        B, T, C = x.size()  # C = n_embd = n_head * head_size

        qkv = self.c_attn(x)    # B,T, 3*n_embd

        q, k, v = qkv.split(self.n_embd, dim=2)    # each : B,T, n_head * head_size

        k = k.view(B, T, self.n_head, self.head_size).transpose(1, 2)    # B, n_head, T, head_size
        q = q.view(B, T, self.n_head, self.head_size).transpose(1, 2)    # ""
        v = v.view(B, T, self.n_head, self.head_size).transpose(1, 2)    # ""

        att = (q @ k.transpose(-2, -1)) * (self.head_size**-0.5)         # B, n_head, T, T
        att = att.masked_fill(self.bias[:T,:T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)

        y = att @ v        # B, n_head, T, head_size
        y = y.transpose(1, 2).contiguous().view(B, T, C)   # B, T , n_embd   (n_embd = n_head * head_size)

        y = self.c_proj(y)
        return y

In [72]:
config = GPT2Config()
h = Attention(config)
# dic = h.state_dict()
# print(h)
# for k,v, in dic.items():
#   print(k,v.shape)

In [73]:
x = torch.randn((32,8,128))
h(x).shape

torch.Size([32, 8, 128])

## Fetch pretrained params from huggingface gpt-2 model

In [74]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

@dataclass
class GPT2Config:
    block_size : int = 1024
    vocab_size : int = 50257
    n_layer : int = 12
    n_head : int = 12
    n_embd : int = 768



# ----------------------------------------------------------------------------------


class GPT2(nn.Module):

    def __init__(self,config):
        super().__init__()

        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size,config.n_embd),
            wpe = nn.Embedding(config.block_size,config.n_embd),

            h = nn.ModuleList([Block(config) for i in range(config.n_layer)]),

            ln_f = nn.LayerNorm(config.n_embd),

        ))

        self.lm_head = nn.Linear(config.n_embd,config.vocab_size, bias=False)



    def forward(self,x,targets= None):

        B,T = x.shape
        assert T<= self.config.block_size   # positional embd table max size = block_size
        tx = self.transformer.wte(x)       #token embedding
        px = self.transformer.wpe(torch.arange(0,T,self.config.block_size,device=device)) #positional embedding

        x = tx+px     # add both

        for block in self.transformer.h:
          x = block(x)

        x = self.transformer.ln_f(x)

        logits = self.lm_head(x)

        if targets is None:
            return logits

        else:
            loss = F.cross_entropy(logits.view(-1,self.config.n_embd),targets.view(-1))
            return logits,loss



    @classmethod
    def from_pretrained(cls, model_type='gpt2'):
        from transformers import GPT2LMHeadModel
        assert model_type == 'gpt2'

        config_args = dict(n_layer=12, n_head=12, n_embd=768, vocab_size=50257, block_size=1024)
        config = GPT2Config(**config_args)
        model = GPT2(config)

        sd = model.state_dict()
        sd_keys = [k for k in sd.keys() if not k.endswith('.attn.bias')]

        model_hf = GPT2LMHeadModel.from_pretrained(model_type)
        sd_hf = model_hf.state_dict()
        sd_keys_hf = [k for k in sd_hf.keys() if not k.endswith(('.attn.masked_bias', '.attn.bias'))]

        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
        assert len(sd_keys_hf) == len(sd_keys)

        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])

        return model






    def generate(self,idx,max_token):
        pass



# ----------------------------------------------------------------------------------



class Block(nn.Module):

    def __init__(self,config):
        super().__init__()
        self.config = config

        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = SelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)


    def forward(self,x):

        x = self.attn(self.ln_1(x)) + x
        x = self.mlp(self.ln_2(x)) + x

        return x



# ----------------------------------------------------------------------------------


class MLP(nn.Module):

    def __init__(self,config):
        super().__init__()
        self.config = config

        self.c_fc = nn.Linear(config.n_embd,4*config.n_embd)
        self.gelu = nn.GELU()
        self.c_proj = nn.Linear(4*config.n_embd,config.n_embd)


    def forward(self,x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)

        return x

# ----------------------------------------------------------------------------------

class SelfAttention(nn.Module):

    def __init__(self, config):
        super().__init__()

        block_size = config.block_size

        self.n_head = n_head = config.n_head
        self.n_embd = n_embd = config.n_embd


        assert n_embd % n_head == 0
        self.head_size = n_embd // n_head

        self.c_attn = nn.Linear(n_embd, 3 * n_embd)     # fan out : n_head * 3 * head_size

        self.c_proj = nn.Linear(n_embd, n_embd)

        self.register_buffer("bias", torch.tril(torch.ones(block_size, block_size)))



    def forward(self, x):
        B, T, C = x.size()  # C = n_embd = n_head * head_size

        qkv = self.c_attn(x)    # B,T, 3*n_embd

        q, k, v = qkv.split(self.n_embd, dim=2)    # each : B,T, n_head * head_size

        k = k.view(B, T, self.n_head, self.head_size).transpose(1, 2)    # B, n_head, T, head_size
        q = q.view(B, T, self.n_head, self.head_size).transpose(1, 2)    # ""
        v = v.view(B, T, self.n_head, self.head_size).transpose(1, 2)    # ""

        att = (q @ k.transpose(-2, -1)) * (self.head_size**-0.5)         # B, n_head, T, T
        att = att.masked_fill(self.bias[:T,:T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)

        y = att @ v        # B, n_head, T, head_size
        y = y.transpose(1, 2).contiguous().view(B, T, C)   # B, T , n_embd   (n_embd = n_head * head_size)

        y = self.c_proj(y)
        return y



In [75]:
config = GPT2Config()
model = GPT2(config)
dic = model.state_dict()


new_d = {}
for k,v, in dic.items():
   new_d[k] = v.shape

In [76]:
for i, (k, v) in enumerate(dic.items()):
    if i < 20 or i > len(dic) - 5:
        print(k, v.shape)

transformer.wte.weight torch.Size([50257, 768])
transformer.wpe.weight torch.Size([1024, 768])
transformer.h.0.ln_1.weight torch.Size([768])
transformer.h.0.ln_1.bias torch.Size([768])
transformer.h.0.attn.bias torch.Size([1024, 1024])
transformer.h.0.attn.c_attn.weight torch.Size([2304, 768])
transformer.h.0.attn.c_attn.bias torch.Size([2304])
transformer.h.0.attn.c_proj.weight torch.Size([768, 768])
transformer.h.0.attn.c_proj.bias torch.Size([768])
transformer.h.0.ln_2.weight torch.Size([768])
transformer.h.0.ln_2.bias torch.Size([768])
transformer.h.0.mlp.c_fc.weight torch.Size([3072, 768])
transformer.h.0.mlp.c_fc.bias torch.Size([3072])
transformer.h.0.mlp.c_proj.weight torch.Size([768, 3072])
transformer.h.0.mlp.c_proj.bias torch.Size([768])
transformer.h.1.ln_1.weight torch.Size([768])
transformer.h.1.ln_1.bias torch.Size([768])
transformer.h.1.attn.bias torch.Size([1024, 1024])
transformer.h.1.attn.c_attn.weight torch.Size([2304, 768])
transformer.h.1.attn.c_attn.bias torch.Si

### Huggingface gpt2 params
```
transformer.wte.weight torch.Size([50257, 768])
transformer.wpe.weight torch.Size([1024, 768])
transformer.h.0.ln_1.weight torch.Size([768])
transformer.h.0.ln_1.bias torch.Size([768])
transformer.h.0.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.0.attn.c_attn.bias torch.Size([2304])
transformer.h.0.attn.c_proj.weight torch.Size([768, 768])
transformer.h.0.attn.c_proj.bias torch.Size([768])
transformer.h.0.ln_2.weight torch.Size([768])
transformer.h.0.ln_2.bias torch.Size([768])
transformer.h.0.mlp.c_fc.weight torch.Size([768, 3072])
transformer.h.0.mlp.c_fc.bias torch.Size([3072])
transformer.h.0.mlp.c_proj.weight torch.Size([3072, 768])
transformer.h.0.mlp.c_proj.bias torch.Size([768])
transformer.h.1.ln_1.weight torch.Size([768])
transformer.h.1.ln_1.bias torch.Size([768])
transformer.h.1.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.1.attn.c_attn.bias torch.Size([2304])
transformer.h.1.attn.c_proj.weight torch.Size([768, 768])
transformer.h.1.attn.c_proj.bias torch.Size([768])
transformer.h.11.mlp.c_proj.bias torch.Size([768])
transformer.ln_f.weight torch.Size([768])
transformer.ln_f.bias torch.Size([768])
lm_head.weight torch.Size([50257, 768])
```


In [77]:
len(new_d)

161

In [78]:
len(real_d)

149

## Generate from model

In [79]:
model = GPT2.from_pretrained()

In [80]:
model.eval()
model.to(device)

GPT2(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (h): ModuleList(
      (0-11): 12 x Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): SelfAttention(
          (c_attn): Linear(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [81]:
import tiktoken

enc = tiktoken.get_encoding('gpt2')

In [82]:
prompt = "Hello World ! I'm LLM"

tokens = enc.encode(prompt)
tokens = torch.tensor([tokens] ,dtype = torch.long)

x = tokens.to(device)

In [83]:
x

tensor([[15496,  2159,  5145,   314,  1101, 27140,    44]], device='cuda:0')

In [84]:
new_max_tokens = 20

while x.size(1) < new_max_tokens:

  with torch.no_grad():
    logits = model(x)

    probs = F.softmax(logits[:,-1,:],dim = -1)

    new_token = torch.multinomial(probs,  num_samples=1)

    x = torch.cat([x,new_token],dim=1)


x

tensor([[15496,  2159,  5145,   314,  1101, 27140,    44,  1440,   329,   262,
         23533,   198,  1450,   351,  1550,   510, 37131,   338,  2714,   423]],
       device='cuda:0')

In [85]:
x = x.view(-1).tolist()

In [86]:
text = enc.decode(x)
text

"Hello World ! I'm LLM four for the medicines\n men with On up MacDonald's held have"